# Categorize Communities and Sentiment Clusters

In [1]:
import pprint
import json
import pathlib
import pandas as pd
import os
import importlib
import networkx as nx
import matplotlib.pyplot as plt
import math
from random import sample
from collections import Counter

In [2]:
from libWill.helpers import *
pp = pprint.PrettyPrinter(indent=4)

### Function to Load Twitter Sentiment Data as Pandas Data Frame

In [10]:
def twitter_premium_as_df(clean_data_path):
    """something like '../../data/clean/' maybe? """
    def listdir_fullpath(d):
        """Function in a functions, whaaaa??!!?!"""
        return [os.path.join(d, f) for f in os.listdir(d)]
    
    clean_data = pathlib.Path(clean_data_path)
    pickles = os.listdir(clean_data)
    full_pickles = listdir_fullpath(clean_data)
    # But this function is doing too much!!
    # Boo hoo 
    return pd.concat([pd.read_pickle(fp) for fp in full_pickles], ignore_index=True)

In [12]:
%time df = twitter_premium_as_df('../../../data/clean/')

CPU times: user 42.8 ms, sys: 278 µs, total: 43.1 ms
Wall time: 40.6 ms


In [29]:
df = df[['created_at','screen_name','full_text','id','geo','lang','mention_ids','mentions','retweet_count','rt_status_id','rt_status_screen_name']]

In [15]:
df.head(2)

,created_at,screen_name,full_text,id,lang,user_favourites_count,user_followers_count,user_friends_count,user_location,user_profile_image_url,...,mentions,mention_ids,quoted_quote_count,quoted_reply_count,quoted_retweet_count,clean_tokens,compound,negative,neutral,positive
0,2019-10-29 11:58:59+00:00,luisgfagundes,RT @CCNMarkets: Starbucks Will Accept Bitcoin ...,1189149629977452544,en,4301,177,461,"Jacksonville Beach, FL",http://pbs.twimg.com/profile_images/9022587817...,...,[CCNMarkets],[1856523530],NaN,NaN,NaN,"[rt, ccnmarkets, starbucks, accept, bitcoin, t...",0.0,0.560,0.440,0.6705
1,2019-10-29 11:58:59+00:00,thefinancialfor,"RT @thefinancialfor: Every year since 2015, #B...",1189149627469193216,en,199,1501,6,"Amsterdam, Holanda",http://pbs.twimg.com/profile_images/1045031709...,...,[thefinancialfor],[2557010554],NaN,NaN,NaN,"[rt, thefinancialfor, every, year, since, bitc...",0.0,0.505,0.495,0.8316


In [19]:
df_network = df[['screen_name','mentions','rt_status_screen_name']]
df_network.head()

,screen_name,mentions,rt_status_screen_name
0,luisgfagundes,[CCNMarkets],CCNMarkets
1,thefinancialfor,[thefinancialfor],thefinancialfor
2,opp4wealth,[RedditBTC],RedditBTC
3,Techtelegraph4,[],NaN
4,tiago60334177,[BitcollarDApp],BitcollarDApp


In [20]:
smallnet = df_network
smallnet.tail()

,screen_name,mentions,rt_status_screen_name
1495,Ratzekatze85,[cryptFund],cryptFund
1496,cardsavvy,[],NaN
1497,jeniffe45644558,[],NaN
1498,DeathByFiat,[beincrypto],NaN
1499,DeathByFiat,[beincrypto],NaN


##Unpacking the mentions column to have multiple rows for each item in the mentions list

In [21]:
smallnet_exploded = smallnet.explode('mentions')

In [23]:
smallnet_exploded

,screen_name,mentions,rt_status_screen_name
0,luisgfagundes,CCNMarkets,CCNMarkets
1,thefinancialfor,thefinancialfor,thefinancialfor
2,opp4wealth,RedditBTC,RedditBTC
3,Techtelegraph4,NaN,NaN
4,tiago60334177,BitcollarDApp,BitcollarDApp
...,...,...,...
1495,Ratzekatze85,cryptFund,cryptFund
1496,cardsavvy,NaN,NaN
1497,jeniffe45644558,NaN,NaN
1498,DeathByFiat,beincrypto,NaN


In [24]:
smallnet_exploded.reset_index(inplace=True,drop=True)

In [25]:
smallnet_exploded.tail()

,screen_name,mentions,rt_status_screen_name
1901,Ratzekatze85,cryptFund,cryptFund
1902,cardsavvy,NaN,NaN
1903,jeniffe45644558,NaN,NaN
1904,DeathByFiat,beincrypto,NaN
1905,DeathByFiat,beincrypto,NaN


In [26]:
edges = list(zip(smallnet_exploded['screen_name'],smallnet_exploded['mentions']))

In [35]:
len(edges)

1399

In [32]:
display(edges[1904],
edges[1905])

('DeathByFiat', 'beincrypto')

('DeathByFiat', 'beincrypto')

In [33]:
edges = [t for t in edges if not any(isinstance(n, float) and math.isnan(n) for n in t)]

In [34]:
len(edges)

1399

In [38]:
#print(edges)

In [36]:
twit_graph = nx.Graph()

In [37]:
twit_graph.add_nodes_from(smallnet['screen_name'])

In [38]:
twit_graph.add_edges_from(edges)

In [39]:
import community
parts = community.best_partition(twit_graph)
values = [parts.get(node) for node in twit_graph.nodes()]

In [40]:
node_list = list(twit_graph.nodes())
community_dict = { n:c for n,c in zip(node_list,values)}
nx.set_node_attributes(twit_graph, community_dict, 'community')

In [41]:
screen_name_list = df['screen_name'].tolist()

In [64]:
# community_df_column = {}
# for key,value in community_dict.items():
#         if key in df['screen_name'].tolist():
#             df['community'] = value
            

In [42]:
df["community"] = df["screen_name"].map(community_dict)

In [43]:
df.tail(2)

,created_at,screen_name,full_text,id,lang,user_favourites_count,user_followers_count,user_friends_count,user_location,user_profile_image_url,...,mention_ids,quoted_quote_count,quoted_reply_count,quoted_retweet_count,clean_tokens,compound,negative,neutral,positive,community
1498,2019-10-18 12:45:20+00:00,DeathByFiat,"Bitcoin Whale Moves 10,000 BTC to Binance, Pot...",1185175026313682950,en,311,95,394,None,http://pbs.twimg.com/profile_images/1160215634...,...,[1021643080208801792],NaN,NaN,NaN,"[bitcoin, whale, move, btc, binance, potential...",0.206,0.794,0.0,-0.3818,601
1499,2019-10-18 12:45:20+00:00,DeathByFiat,"Bitcoin Whale Moves 10,000 BTC to Binance, Pot...",1185175026313682950,en,311,95,394,None,http://pbs.twimg.com/profile_images/1160215634...,...,[1021643080208801792],NaN,NaN,NaN,"[bitcoin, whale, move, btc, binance, potential...",0.206,0.794,0.0,-0.3818,601


In [44]:
nx.number_of_nodes(twit_graph)

1724

In [45]:
nx.number_of_edges(twit_graph)

In [85]:
#

In [83]:
# drop_nodes = []
# for i in twit_graph.nodes:
#     if twit_graph.degree(i) < 3:
#         drop_nodes.append(i)

In [84]:
#twit_graph.remove_nodes_from(drop_nodes)

In [46]:
nx.number_of_nodes(twit_graph)

1724

In [48]:
import community
parts = community.best_partition(twit_graph)
values = [parts.get(node) for node in twit_graph.nodes()]

In [631]:
print(community.__version__)

0.13


In [633]:
#! dir C:\Users\mferb\Anaconda3\envs\project2\lib\site-packages\community

In [634]:
#! conda list community

In [49]:
node_list = list(twit_graph.nodes())
community_dict = { n:c for n,c in zip(node_list,values)}
nx.set_node_attributes(twit_graph, community_dict, 'community')

In [50]:
from bokeh.io import show, save, output_file 
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral8
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
import holoviews as hv


In [51]:
hv.extension('bokeh')

In [120]:

import random
sizes = [random.randrange(8, 20, 1) for i in range(nx.number_of_nodes(twit_graph))]

```
import networkx as nx
import matplotlib.pyplot as plt

g = nx.Graph()
g.add_edges_from([(1,2), (2,3), (2,4), (3,4)])

d = nx.degree(g)

nx.draw(g, nodelist=d.keys(), node_size=[v * 100 for v in d.values()])
plt.show()
```

In [135]:
%opts Graph [width=1200 height=800]
padding = dict(x=(-1.1, 1.1), y=(-1.1, 1.1))
#cmap=plt.cm.Blues
#[v * 100 for v in d.values()]
# g = nx.Graph(twit_graph)
# d = dict(twit_graph.degree()).values()
# sorta worked # hv.dim('size')*20
%time twitter_graph = hv.Graph.from_networkx(twit_graph, nx.layout.spring_layout)\
                .redim.range(**padding)\
                .options(color_index='community',node_alpha=0.5, node_size=20,edge_line_width=1,edge_line_color='black',cmap='Category20', colorbar=True, axiswise=True)
#hv.save(twitter_graph, 'holoview_kamada_layout.html')                                        
#hv.save(Graph, 'holoview_test.html')

CPU times: user 18.3 s, sys: 0 ns, total: 18.3 s
Wall time: 18.4 s


In [136]:
twitter_graph

:Graph   [start,end]

In [110]:
%opts Graph [width=1200 height=900]
padding = dict(x=(-1.1, 1.1), y=(-1.1, 1.1))
#cmap=plt.cm.Blues
#[v * 100 for v in d.values()]
# g = nx.Graph(twit_graph)
# d = dict(twit_graph.degree()).values()
# kwargs={'weight': 'weight'}
%time twitter_circ_graph = hv.Graph.from_networkx(twit_graph, nx.layout.fruchterman_reingold_layout,)\
                .redim.range(**padding)\
                .options(color_index='community',node_alpha=0.5, node_size=20,edge_line_width=1,edge_line_color='#333333',cmap='Category20', colorbar=True, axiswise=True)
#hv.save(twitter_graph, 'holoview_kamada_layout.html')                                        
#hv.save(Graph, 'holoview_test.html')

CPU times: user 19.8 s, sys: 0 ns, total: 19.8 s
Wall time: 19.9 s


In [111]:
twitter_circ_graph

:Graph   [start,end]

In [434]:
# #hover = HoverTool(tooltips=[('screen_name', twit_graph.nodes)])
                  
# plot = figure(title="5000 Tweets Test",
#               width=800,height=600,
#               tools=['box_zoom','pan','wheel_zoom','reset'], x_range=(-1.1,1.1), y_range=(-1.1,1.1))

# graph = from_networkx(twit_graph, nx.spring_layout, scale=3, center=(0,0))
# # graph.node_renderer.data_source = values

# graph.node_renderer.data_source.data['community'] = [i[1]['community'] for i in twit_graph.nodes(data=True)]
# graph.node_renderer.glyph = Circle(
#     size=10, 
#     fill_color=linear_cmap('community', 'Spectral8', 0, len(twit_graph.nodes()))
# )

# plot.renderers.append(graph)

# #output_file("5000_tweet_test_community.html")
# show(plot)

In [507]:
from holoviews.operation.datashader import datashade, bundle_graph


In [540]:
bundled = bundle_graph(twitter_graph)
bundled

:Graph   [start,end]

:Graph   [start,end]

In [625]:
sorted(twit_graph.degree, key=lambda x: x[1], reverse=True)

[('APompliano', 80),
 ('Rhythmtrader', 65),
 ('realDonaldTrump', 36),
 ('CalvinAyre', 31),
 ('TheCryptoconomy', 30),
 ('Bitcoin_Beyond', 29),
 ('Cointelegraph', 28),
 ('saifedean', 27),
 ('Xentagz', 25),
 ('coindesk', 24),
 ('DesiCryptoHodlr', 23),
 ('pierre_rochard', 22),
 ('bitcoin_clara', 22),
 ('Twanomatic2', 20),
 ('kurtwuckertjr', 19),
 ('bitcoin_ug', 19),
 ('lopp', 19),
 ('NIXwhale', 19),
 ('ErikVoorhees', 19),
 ('Bitboy_Crypto', 19),
 ('aantonop', 19),
 ('CoinstorageGuru', 18),
 ('danheld', 18),
 ('Ragnarly', 18),
 ('american_hodl', 18),
 ('alistairmilne', 18),
 ('Bitcoin', 18),
 ('MichellePhan', 18),
 ('PeterSchiff', 18),
 ('Emperor_YZ', 17),
 ('Josh_Rager', 17),
 ('Poetry4Bitcoin', 17),
 ('dergigi', 17),
 ('Hermetec', 17),
 ('_blockandchain_', 17),
 ('bitstein', 17),
 ('TuurDemeester', 17),
 ('HotepJesus', 17),
 ('Capfolio', 17),
 ('EntangleIT', 16),
 ('eternalcrypto1', 16),
 ('Bitcoin_org_', 15),
 ('JimmyNLose', 15),
 ('_omohoje', 15),
 ('M_ednut', 15),
 ('johnkim77', 15),
 

In [163]:
community = nx.get_node_attributes(twit_graph,'Community')

In [175]:
largest_community = Counter(community['Bitcoin'])
largest_community.most_common()

[(8, 97),
 (30, 94),
 (7, 84),
 (5, 75),
 (112, 72),
 (0, 71),
 (3, 70),
 (27, 67),
 (60, 67),
 (21, 62),
 (24, 62),
 (35, 58),
 (57, 56),
 (31, 52),
 (9, 50),
 (14, 48),
 (205, 43),
 (12, 40),
 (33, 35),
 (74, 31),
 (11, 25),
 (278, 23),
 (22, 22),
 (37, 19),
 (64, 17),
 (130, 16),
 (1, 14),
 (15, 12),
 (18, 12),
 (125, 12),
 (56, 10),
 (114, 9),
 (174, 9),
 (299, 9),
 (2, 8),
 (50, 8),
 (13, 7),
 (29, 7),
 (49, 7),
 (68, 7),
 (106, 7),
 (163, 7),
 (190, 7),
 (281, 7),
 (372, 7),
 (23, 6),
 (48, 6),
 (66, 6),
 (71, 6),
 (72, 6),
 (113, 6),
 (135, 6),
 (176, 6),
 (244, 6),
 (266, 6),
 (331, 6),
 (19, 5),
 (20, 5),
 (26, 5),
 (38, 5),
 (42, 5),
 (46, 5),
 (53, 5),
 (54, 5),
 (69, 5),
 (93, 5),
 (103, 5),
 (120, 5),
 (122, 5),
 (133, 5),
 (151, 5),
 (154, 5),
 (184, 5),
 (294, 5),
 (459, 5),
 (4, 4),
 (16, 4),
 (36, 4),
 (39, 4),
 (45, 4),
 (67, 4),
 (83, 4),
 (85, 4),
 (91, 4),
 (126, 4),
 (138, 4),
 (139, 4),
 (140, 4),
 (149, 4),
 (159, 4),
 (167, 4),
 (180, 4),
 (197, 4),
 (204, 4),


In [178]:
print(len(community['Bitcoin']))

2632


In [176]:
nodes = list(twit_graph.nodes)

In [182]:
print(nodes[3])

Techcenterr


In [36]:
isolates = nx.isolates(twit_graph)

In [38]:
list(isolates)

['BtcEthereum',
 'RockstaTrading',
 'squatch_crypto',
 'FinTechNames',
 'PatriotCrypto',
 'BitBullCapital',
 'duniajalanjalan',
 'Etherbankingita',
 'Franzferdinan57',
 'ICOResearchGRP',
 'SportsbookBTC',
 'BitcoinCatz',
 'Crypt0Constella',
 'VoCtpgC8Ki241Rg',
 'KriptoKanalim',
 'webnowcompany',
 'coinbureau',
 'mospretmen',
 'ALTcoin_Ninjas',
 'bitlyfool',
 'retiredin2016',
 'HashDeploy',
 'TheeSeanH',
 'Lobaoal',
 'ShillHater',
 'ib_cpa',
 'AntonioSerbia',
 'SnowflakesAwwww',
 'Blacktradelines',
 'cryptogarten',
 'crypto__mak',
 'jadwahab',
 'HydaxE',
 'coinsonnews',
 'BizSoftwareEdu',
 'thecryptorep',
 'Ursalacher',
 'AinslieWealth',
 'TheSh0tC4ll3r',
 'InvestForward18',
 'icoinmarket1',
 'dragonwolftech',
 'robertdolci',
 'NachoThome',
 'DarkNodeGuru',
 'foxesden34',
 'BigCheds',
 'help_me_plzplz_',
 'News_1jl4',
 'Teokanistras',
 'PaulESamson',
 'AvyLorenCohen',
 'grbroker',
 'Cryptonetwealth',
 'GetCoinJar',
 'URBitcoinwear',
 'blockchainoshi',
 'bitcoin_jar',
 'FreeCryptoDeals',

In [71]:
twit_graph.remove_nodes_from(list(nx.isolates(twit_graph)))

In [72]:
nx.number_of_nodes(twit_graph)

3156

[('APompliano', 84),
 ('Rhythmtrader', 54),
 ('Cointelegraph', 38),
 ('jack', 29),
 ('100trillionUSD', 26),
 ('Bitcoin', 22),
 ('PeterSchiff', 21),
 ('qhfofficial', 20),
 ('BitCoin_Invest_', 20),
 ('Poetry4Bitcoin', 17),
 ('coindesk', 16),
 ('Hermetec', 16),
 ('Satoshis_D', 15),
 ('SammarA12281909', 13),
 ('Bitcoin_Beyond', 13),
 ('codeyisfun', 12),
 ('CalvinAyre', 11),
 ('TheCryptoconomy', 11),
 ('popesta', 11),
 ('realDonaldTrump', 11),
 ('Bakkt', 11),
 ('PeterMcCormack', 11),
 ('american_hodl', 11),
 ('CryptoOz', 10),
 ('Hanakookie1', 10),
 ('NemesisBSV', 10),
 ('Cripto_Bitcoin', 10),
 ('BTCTN', 10),
 ('StormIsUponUs', 10),
 ('ibmlahamad', 9),
 ('BitcoinEcon', 9),
 ('Coinicarus', 9),
 ('YobitsVision', 9),
 ('LomahCrypto', 9),
 ('Mihalbig', 9),
 ('Xentagz', 9),
 ('MZietzke', 9),
 ('Anders_', 9),
 ('trylolli', 9),
 ('HotepJesus', 9),
 ('HonorAfricans', 9),
 ('theonevortex', 8),
 ('InvestCrypForex', 8),
 ('Emperor_YZ', 8),
 ('channelBSV', 8),
 ('mindstatex', 8),
 ('pierre_rochard', 8),

In [85]:
nx.average_degree_connectivity(twit_graph)

{1: 6.369631236442516,
 3: 3.8732943469785575,
 2: 7.258947368421053,
 4: 4.375,
 9: 4.981481481481482,
 6: 2.8015873015873014,
 8: 3.46875,
 5: 2.9348837209302325,
 7: 4.333333333333333,
 84: 3.0357142857142856,
 11: 3.9220779220779223,
 10: 3.8333333333333335,
 13: 6.230769230769231,
 16: 2.875,
 15: 4.8,
 21: 6.761904761904762,
 17: 5.823529411764706,
 38: 1.9210526315789473,
 54: 1.5740740740740742,
 29: 2.0,
 22: 3.3181818181818183,
 20: 1.025,
 12: 1.8333333333333333,
 26: 2.923076923076923}

In [93]:
edges_for_node_x = twit_graph.degree('Rhythmtrader')

In [94]:
print(edges_for_node_x)

54


In [100]:
print(len(drop_nodes))

2780


In [1]:
print(len(community_dict))

NameError: name 'community_dict' is not defined